# Fine-tuning, Inference, and Evaluation with NVIDIA NeMo Microservices and NIM

### Introduction

This notebook covers the following workflows:
- Creating a dataset and uploading files for customizing and evaluating models
- Running inference on base and customized models
- Customizing and evaluating models, comparing metrics between base models and fine-tuned models
- Running a safety check and evaluating a model using Guardrails


## Prerequisites

### Deploy NeMo Microservices
Ensure the NeMo Microservices platform is up and running, including the model downloading step for `meta/llama-3.1-8b-instruct`. Please refer to the [installation guide](https://aire.gitlab-master-pages.nvidia.com/microservices/documentation/latest/nemo-microservices/latest-internal/set-up/deploy-as-platform/index.html) for instructions.

You can verify the `meta/llama-3.1-8b-instruct` is deployed by querying the NIM endpoint. The response should include a model with an `id` of `meta/llama-3.1-8b-instruct`.

```bash
# URL to NeMo deployment management service
export NEMO_URL="http://nemo.test"

curl -X GET "$NEMO_URL/v1/models" \
  -H "Accept: application/json"
```

### Set up Developer Environment
Set up your development environment on your machine. The project uses `uv` to manage Python dependencies. From the root of the project, install dependencies and create your virtual environment:

```bash
uv sync --extra dev
uv pip install -e .
source .venv/bin/activate
```

### Build Llama Stack Image
Build the Llama Stack image using the virtual environment you just created. For local development, set `LLAMA_STACK_DIR` to ensure your local code is use in the image. To use the production version of `llama-stack`, omit `LLAMA_STACK_DIR`.

```bash
LLAMA_STACK_DIR=$(pwd) llama stack build --distro nvidia --image-type venv
```

## Setup

1. Update the following variables in [config.py](./config.py) with your deployment URLs and API keys. The other variables are optional. You can update these to organize the resources created by this notebook.
```python
# (Required) NeMo Microservices URLs
NDS_URL = "" # NeMo Data Store
NEMO_URL = "" # Other NeMo Microservices (Customizer, Evaluator, Guardrails)
NIM_URL = "" # NIM

# (Required) Hugging Face Token
HF_TOKEN = ""
```

2. Set environment variables used by each service.

In [1]:
import os
from docs.notebooks.nvidia.beginner_e2e.config import *

# Metadata associated with Datasets and Customization Jobs
os.environ["NVIDIA_DATASET_NAMESPACE"] = NAMESPACE
os.environ["NVIDIA_PROJECT_ID"] = PROJECT_ID

# Inference env vars
os.environ["NVIDIA_BASE_URL"] = NIM_URL

# Data Store env vars
os.environ["NVIDIA_DATASETS_URL"] = NEMO_URL

# Customizer env vars
os.environ["NVIDIA_CUSTOMIZER_URL"] = NEMO_URL
os.environ["NVIDIA_OUTPUT_MODEL_DIR"] = CUSTOMIZED_MODEL_DIR

# Evaluator env vars
os.environ["NVIDIA_EVALUATOR_URL"] = NEMO_URL

# Guardrails env vars
os.environ["GUARDRAILS_SERVICE_URL"] = NEMO_URL


3. Initialize the HuggingFace API client. Here, we use NeMo Data Store as the endpoint the client will invoke.

In [ ]:
from huggingface_hub import HfApi
import json
import pprint
import requests
from time import sleep, time

os.environ["HF_ENDPOINT"] = f"{NDS_URL}/v1/hf"
os.environ["HF_TOKEN"] = HF_TOKEN

hf_api = HfApi(endpoint=os.environ.get("HF_ENDPOINT"), token=os.environ.get("HF_TOKEN"))

4. Initialize the Llama Stack client using the NVIDIA provider.

In [ ]:
from llama_stack.core.library_client import LlamaStackAsLibraryClient

client =  LlamaStackAsLibraryClient("nvidia")
client.initialize()

5. Define a few helper functions we'll use later that wait for async jobs to complete.

In [4]:
from llama_stack.apis.common.job_types import JobStatus

def wait_customization_job(job_id: str, polling_interval: int = 30, timeout: int = 3600):
    start_time = time()

    response = client.post_training.job.status(job_uuid=job_id)
    job_status = response.status

    print(f"Waiting for Customization job {job_id} to finish.")
    print(f"Job status: {job_status} after {time() - start_time} seconds.")

    while job_status in [JobStatus.scheduled.value, JobStatus.in_progress.value]:
        sleep(polling_interval)
        response = client.post_training.job.status(job_uuid=job_id)
        job_status = response.status

        print(f"Job status: {job_status} after {time() - start_time} seconds.")

        if time() - start_time > timeout:
            raise RuntimeError(f"Customization Job {job_id} took more than {timeout} seconds.")
        
    return job_status

def wait_eval_job(benchmark_id: str, job_id: str, polling_interval: int = 10, timeout: int = 6000):
    start_time = time()
    job_status = client.eval.jobs.status(benchmark_id=benchmark_id, job_id=job_id)

    print(f"Waiting for Evaluation job {job_id} to finish.")
    print(f"Job status: {job_status} after {time() - start_time} seconds.")

    while job_status.status in [JobStatus.scheduled.value, JobStatus.in_progress.value]:
        sleep(polling_interval)
        job_status = client.eval.jobs.status(benchmark_id=benchmark_id, job_id=job_id)

        print(f"Job status: {job_status} after {time() - start_time} seconds.")

        if time() - start_time > timeout:
            raise RuntimeError(f"Evaluation Job {job_id} took more than {timeout} seconds.")

    return job_status

# When creating a customized model, NIM asynchronously loads the model in its model registry.
# After this, we can run inference on the new model. This helper function waits for NIM to pick up the new model.
def wait_nim_loads_customized_model(model_id: str, polling_interval: int = 10, timeout: int = 300):
    found = False
    start_time = time()

    print(f"Checking if NIM has loaded customized model {model_id}.")

    while not found:
        sleep(polling_interval)

        response = requests.get(f"{NIM_URL}/v1/models")
        if model_id in [model["id"] for model in response.json()["data"]]:
            found = True
            print(f"Model {model_id} available after {time() - start_time} seconds.")
            break
        else:
            print(f"Model {model_id} not available after {time() - start_time} seconds.")

    if not found:
        raise RuntimeError(f"Model {model_id} not available after {timeout} seconds.")

    assert found, f"Could not find model {model_id} in the list of available models."
            

## Upload Dataset Using the HuggingFace Client

Start by creating a dataset with the `sample_squad_data` files. This data is pulled from the Stanford Question Answering Dataset (SQuAD) reading comprehension dataset, consisting of questions posed on a set of Wikipedia articles, where the answer to every question is a segment of text from the corresponding passage, or the question is unanswerable.

In [6]:
sample_squad_dataset_name = "sample-squad-test"
repo_id = f"{NAMESPACE}/{sample_squad_dataset_name}"

In [7]:
# Create the repo
response = hf_api.create_repo(repo_id, repo_type="dataset")

In [ ]:
# Upload the files from the local folder
hf_api.upload_folder(
    folder_path="./sample_data/sample_squad_data/training",
    path_in_repo="training",
    repo_id=repo_id,
    repo_type="dataset",
)
hf_api.upload_folder(
    folder_path="./sample_data/sample_squad_data/validation",
    path_in_repo="validation",
    repo_id=repo_id,
    repo_type="dataset",
)
hf_api.upload_folder(
    folder_path="./sample_data/sample_squad_data/testing",
    path_in_repo="testing",
    repo_id=repo_id,
    repo_type="dataset",
)

In [ ]:
# Create the dataset
response = client.datasets.register(
    purpose="post-training/messages",
    dataset_id=sample_squad_dataset_name,
    source={
        "type": "uri",
        "uri": f"hf://datasets/{repo_id}"
    },
    metadata={
        "format": "json",
        "description": "Test sample_squad_data dataset for NVIDIA E2E notebook",
        "provider_id": "nvidia",
    }
)
print(response)

In [ ]:
# Check the files URL
response = requests.get(
    url=f"{NEMO_URL}/v1/datasets/{NAMESPACE}/{sample_squad_dataset_name}",
)
assert response.status_code in (200, 201), f"Status Code {response.status_code} Failed to fetch dataset {response.text}"

dataset_obj = response.json()
print("Files URL:", dataset_obj["files_url"])
assert dataset_obj["files_url"] == f"hf://datasets/{repo_id}"

## Inference

We'll use an entry from the `sample_squad_data` test data to verify we can run inference using NVIDIA NIM.

In [ ]:
import json
import pprint

with open("./sample_data/sample_squad_data/testing/testing.jsonl", "r") as f:
    examples = [json.loads(line) for line in f]

# Get the user prompt from the last example
sample_prompt = examples[-1]["prompt"]
pprint.pprint(sample_prompt)

In [ ]:
# Test inference
response = client.inference.chat_completion(
    messages=[
        {"role": "user", "content": sample_prompt}
    ],
    model_id=BASE_MODEL,
    sampling_params={
        "max_tokens": 20,
        "strategy": {
            "type": "top_p",
            "temperature": 0.7,
            "top_p": 0.9
        }
    }
)
print(f"Inference response: {response.completion_message.content}")

## Evaluation


To run an Evaluation, we'll first register a benchmark. A benchmark corresponds to an Evaluation Config in NeMo Evaluator, which contains the metadata to use when launching an Evaluation Job. Here, we'll create a benchmark that uses the testing file uploaded in the previous step. 

In [19]:
benchmark_id = "test-eval-config"

In [20]:
simple_eval_config = {
    "benchmark_id": benchmark_id,
    "dataset_id": "",
    "scoring_functions": [],
    "metadata": {
        "type": "custom",
        "params": {"parallelism": 8},
        "tasks": {
            "qa": {
                "type": "completion",
                "params": {
                    "template": {
                        "prompt": "{{prompt}}",
                        "max_tokens": 20,
                        "temperature": 0.7,
                        "top_p": 0.9,
                    },
                },
                "dataset": {"files_url": f"hf://datasets/{repo_id}/testing/testing.jsonl"},
                "metrics": {
                    "bleu": {
                        "type": "bleu",
                        "params": {"references": ["{{ideal_response}}"]},
                    },
                    "string-check": {
                        "type": "string-check",
                        "params": {"check": ["{{ideal_response | trim}}", "equals", "{{output_text | trim}}"]},
                    },
                },
            }
        }
    }
}

In [ ]:
# Register a benchmark, which creates an Evaluation Config
response = client.benchmarks.register(
    benchmark_id=benchmark_id,
    dataset_id=repo_id,
    scoring_functions=simple_eval_config["scoring_functions"],
    metadata=simple_eval_config["metadata"]
)
print(f"Created benchmark {benchmark_id}")

In [ ]:
# Launch a simple evaluation with the benchmark
response = client.eval.run_eval(
    benchmark_id=benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": BASE_MODEL,
            "sampling_params": {}
        }
    }
)
job_id = response.model_dump()["job_id"]
print(f"Created evaluation job {job_id}")

In [ ]:
# Wait for the job to complete
job = wait_eval_job(benchmark_id=benchmark_id, job_id=job_id, polling_interval=5, timeout=600)

In [ ]:
print(f"Job {job_id} status: {job.status}")

In [ ]:
job_results = client.eval.jobs.retrieve(benchmark_id=benchmark_id, job_id=job_id)
print(f"Job results: {json.dumps(job_results.model_dump(), indent=2)}")

In [ ]:
# Extract bleu score and assert it's within range
initial_bleu_score = job_results.scores[benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["bleu"]["scores"]["corpus"]["value"]
print(f"Initial bleu score: {initial_bleu_score}")

assert initial_bleu_score >= 2

In [ ]:
# Extract accuracy and assert it's within range
initial_accuracy_score = job_results.scores[benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["string-check"]["scores"]["string-check"]["value"]
print(f"Initial accuracy: {initial_accuracy_score}")

assert initial_accuracy_score >= 0

## Customization

Now that we've established our baseline Evaluation metrics, we'll customize a model using our training data uploaded previously.

In [ ]:
# Start the customization job
response = client.post_training.supervised_fine_tune(
    job_uuid="",
    model=BASE_MODEL,
    training_config={
        "n_epochs": 2,
        "data_config": {
            "batch_size": 16,
            "dataset_id": sample_squad_dataset_name,
        },
        "optimizer_config": {
            "lr": 0.0001,
        }
    },
    algorithm_config={
        "type": "LoRA",
        "adapter_dim": 16,
        "adapter_dropout": 0.1,
        "alpha": 16,
        # NOTE: These fields are required, but not directly used by NVIDIA
        "rank": 8,
        "lora_attn_modules": [],
        "apply_lora_to_mlp": True,
        "apply_lora_to_output": False
    },
    hyperparam_search_config={},
    logger_config={},
    checkpoint_dir="",
)

job_id = response.job_uuid
print(f"Created job with ID: {job_id}")

In [ ]:
# Wait for the job to complete
job_status = wait_customization_job(job_id=job_id)

In [ ]:
print(f"Job {job_id} status: {job_status}")

After the fine-tuning job succeeds, we can't immediately run inference on the customized model. In the background, NIM will load newly-created models and make them available for inference. This process typically takes < 5 minutes - here, we wait for our customized model to be picked up before attempting to run inference.

In [ ]:
# Check that the customized model has been picked up by NIM;
# We allow up to 5 minutes for the LoRA adapter to be loaded
wait_nim_loads_customized_model(model_id=CUSTOMIZED_MODEL_DIR)

At this point, NIM can run inference on the customized model. However, to use the Llama Stack client to run inference, we need to explicitly register the model first.

In [ ]:
# Check that inference with the new model works
from llama_stack.apis.models.models import ModelType

# First, register the customized model
client.models.register(
    model_id=CUSTOMIZED_MODEL_DIR,
    model_type=ModelType.llm,
    provider_id="nvidia",
)

response = client.inference.completion(
    content="Complete the sentence using one word: Roses are red, violets are ",
    stream=False,
    model_id=CUSTOMIZED_MODEL_DIR,
    sampling_params={
        "strategy": {
            "type": "top_p",
            "temperature": 0.7,
            "top_p": 0.9
        },
        "max_tokens": 20,
    },
)
print(f"Inference response: {response.content}")

## Evaluate Customized Model
Now that we've customized the model, let's run another Evaluation to compare its performance with the base model.

In [ ]:
# Launch a simple evaluation with the same benchmark with the customized model
response = client.eval.run_eval(
    benchmark_id=benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": CUSTOMIZED_MODEL_DIR,
            "sampling_params": {}
        }
    }
)
job_id = response.model_dump()["job_id"]
print(f"Created evaluation job {job_id}")

In [ ]:
# Wait for the job to complete
customized_model_job = wait_eval_job(benchmark_id=benchmark_id, job_id=job_id, polling_interval=5, timeout=600)

In [ ]:
print(f"Job {job_id} status: {customized_model_job.status}")

In [ ]:
customized_model_job_results = client.eval.jobs.retrieve(benchmark_id=benchmark_id, job_id=job_id)
print(f"Job results: {json.dumps(customized_model_job_results.model_dump(), indent=2)}")

In [ ]:
# Extract bleu score and assert it's within range
customized_bleu_score = customized_model_job_results.scores[benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["bleu"]["scores"]["corpus"]["value"]
print(f"Customized bleu score: {customized_bleu_score}")

assert customized_bleu_score >= 35

In [ ]:
# Extract accuracy and assert it's within range
customized_accuracy_score = customized_model_job_results.scores[benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["string-check"]["scores"]["string-check"]["value"]
print(f"Initial accuracy: {customized_accuracy_score}")

assert customized_accuracy_score >= 0.45

We expect to see an improvement in the bleu score and accuracy in the customized model's evaluation results.

In [ ]:
# Ensure the customized model evaluation is better than the original model evaluation
print(f"customized_bleu_score - initial_bleu_score: {customized_bleu_score - initial_bleu_score}")
assert (customized_bleu_score - initial_bleu_score) >= 27

print(f"customized_accuracy_score - initial_accuracy_score: {customized_accuracy_score - initial_accuracy_score}")
assert (customized_accuracy_score - initial_accuracy_score) >= 0.4

## Upload Chat Dataset Using the HuggingFace Client
Repeat the fine-tuning and evaluation workflow with a chat-style dataset, which has a list of `messages` instead of a `prompt` and `completion`.

In [29]:
sample_squad_messages_dataset_name = "test-squad-messages-dataset"
repo_id = f"{NAMESPACE}/{sample_squad_messages_dataset_name}"

In [30]:
# Create the repo
res = hf_api.create_repo(repo_id, repo_type="dataset")

In [ ]:
# Upload the files from the local folder
hf_api.upload_folder(
    folder_path="./sample_data/sample_squad_messages/training",
    path_in_repo="training",
    repo_id=repo_id,
    repo_type="dataset",
)
hf_api.upload_folder(
    folder_path="./sample_data/sample_squad_messages/validation",
    path_in_repo="validation",
    repo_id=repo_id,
    repo_type="dataset",
)
hf_api.upload_folder(
    folder_path="./sample_data/sample_squad_messages/testing",
    path_in_repo="testing",
    repo_id=repo_id,
    repo_type="dataset",
)

In [ ]:
# Create the dataset
response = client.datasets.register(
    purpose="post-training/messages",
    dataset_id=sample_squad_messages_dataset_name,
    source={
        "type": "uri",
        "uri": f"hf://datasets/{repo_id}"
    },
    metadata={
        "format": "json",
        "description": "Test sample_squad_messages dataset for NVIDIA E2E notebook",
        "provider_id": "nvidia",
    }
)
print(response)

In [ ]:
# Check the files URL
response = requests.get(
    url=f"{NEMO_URL}/v1/datasets/{NAMESPACE}/{sample_squad_messages_dataset_name}",
)
assert response.status_code in (200, 201), f"Status Code {response.status_code} Failed to fetch dataset {response.text}"
dataset_obj = response.json()
print("Files URL:", dataset_obj["files_url"])
assert dataset_obj["files_url"] == f"hf://datasets/{repo_id}"

## Inference with chat/completions
We'll use an entry from the `sample_squad_messages` test data to verify we can run inference using NVIDIA NIM.

In [ ]:
with open("./sample_data/sample_squad_messages/testing/testing.jsonl", "r") as f:
    examples = [json.loads(line) for line in f]

# get the user and assistant messages from the last example
sample_messages = examples[-1]["messages"][:-1]
pprint.pprint(sample_messages)

In [ ]:
# Test inference
response = client.inference.chat_completion(
    messages=sample_messages,
    model_id=BASE_MODEL,
    sampling_params={
        "max_tokens": 20,
        "strategy": {
            "type": "top_p",
            "temperature": 0.7,
            "top_p": 0.9
        }
    }
)
assert response.completion_message.content is not None
print(f"Inference response: {response.completion_message.content}")

## Evaluate with chat dataset
We'll register a new benchmark that uses the chat-style testing file uploaded previously.

In [36]:
benchmark_id = "test-eval-config-chat"

In [37]:
# Register a benchmark, which creates an Eval Config
simple_eval_config = {
    "benchmark_id": benchmark_id,
    "dataset_id": "",
    "scoring_functions": [],
    "metadata": {
        "type": "custom",
        "params": {"parallelism": 8},
        "tasks": {
            "qa": {
                "type": "completion",
                "params": {
                    "template": {
                        "messages": [
                            {"role": "{{item.messages[0].role}}", "content": "{{item.messages[0].content}}"},
                            {"role": "{{item.messages[1].role}}", "content": "{{item.messages[1].content}}"},
                        ],
                        "max_tokens": 20,
                        "temperature": 0.7,
                        "top_p": 0.9,
                    },
                },
                "dataset": {"files_url": f"hf://datasets/{repo_id}/testing/testing.jsonl"},
                "metrics": {
                    "bleu": {
                        "type": "bleu",
                        "params": {"references": ["{{item.messages[2].content | trim}}"]},
                    },
                    "string-check": {
                        "type": "string-check",
                        "params": {"check": ["{{item.messages[2].content}}", "equals", "{{output_text | trim}}"]},
                    },
                },
            }
        }
    }
}

In [ ]:
response = client.benchmarks.register(
    benchmark_id=benchmark_id,
    dataset_id=repo_id,
    scoring_functions=simple_eval_config["scoring_functions"],
    metadata=simple_eval_config["metadata"]
)
print(f"Created benchmark {benchmark_id}")

In [ ]:
# Launch a simple evaluation with the benchmark
response = client.eval.run_eval(
    benchmark_id=benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": BASE_MODEL,
            "sampling_params": {}
        }
    }
)
job_id = response.model_dump()["job_id"]
print(f"Created evaluation job {job_id}")

In [ ]:
# Wait for the job to complete
job = wait_eval_job(benchmark_id=benchmark_id, job_id=job_id, polling_interval=5, timeout=600)

In [ ]:
print(f"Job {job_id} status: {job.status}")

In [ ]:
job_results = client.eval.jobs.retrieve(benchmark_id=benchmark_id, job_id=job_id)
print(f"Job results: {json.dumps(job_results.model_dump(), indent=2)}")

In [ ]:
# Extract bleu score and assert it's within range
initial_bleu_score = job_results.scores[benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["bleu"]["scores"]["corpus"]["value"]
print(f"Initial bleu score: {initial_bleu_score}")

assert initial_bleu_score >= 12

In [ ]:
# Extract accuracy and assert it's within range
initial_accuracy_score = job_results.scores[benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["string-check"]["scores"]["string-check"]["value"]
print(f"Initial accuracy: {initial_accuracy_score}")

assert initial_accuracy_score >= 0.2

## Customization with chat dataset

Now that we've established our baseline Evaluation metrics for the chat-style dataset, we'll customize a model using our training data uploaded previously.

In [ ]:
customized_chat_model_name = "test-messages-model"
customized_chat_model_version = "v1"
customized_chat_model_dir = f"{NAMESPACE}/{customized_chat_model_name}@{customized_chat_model_version}"

# NOTE: The output model name is derived from the environment variable. We need to re-initialize the client
# here so the Post Training API picks up the updated value.
os.environ["NVIDIA_OUTPUT_MODEL_DIR"] = customized_chat_model_dir
client.initialize()

In [ ]:
response = client.post_training.supervised_fine_tune(
    job_uuid="",
    model=BASE_MODEL,
    training_config={
        "n_epochs": 2,
        "data_config": {
            "batch_size": 16,
            "dataset_id": sample_squad_messages_dataset_name,
        },
        "optimizer_config": {
            "lr": 0.0001,
        }
    },
    algorithm_config={
        "type": "LoRA",
        "adapter_dim": 16,
        "adapter_dropout": 0.1,
        "alpha": 16,
        # NOTE: These fields are required, but not directly used by NVIDIA
        "rank": 8,
        "lora_attn_modules": [],
        "apply_lora_to_mlp": True,
        "apply_lora_to_output": False
    },
    hyperparam_search_config={},
    logger_config={},
    checkpoint_dir="",
)

job_id = response.job_uuid
print(f"Created job with ID: {job_id}")

In [ ]:
job = wait_customization_job(job_id=job_id, polling_interval=30, timeout=3600)

In [ ]:
print(f"Job {job_id} status: {job_status}")

In [ ]:
# Check that the customized model has been picked up by NIM;
# We allow up to 5 minutes for the LoRA adapter to be loaded
wait_nim_loads_customized_model(model_id=customized_chat_model_dir)

In [ ]:
# Check that inference with the new customized model works
from llama_stack.apis.models.models import ModelType

# First, register the customized model
client.models.register(
    model_id=customized_chat_model_dir,
    model_type=ModelType.llm,
    provider_id="nvidia",
)

response = client.inference.completion(
    content="Complete the sentence using one word: Roses are red, violets are ",
    stream=False,
    model_id=customized_chat_model_dir,
    sampling_params={
        "strategy": {
            "type": "top_p",
            "temperature": 0.7,
            "top_p": 0.9
        },
        "max_tokens": 20,
    },
)
print(f"Inference response: {response.content}")

In [ ]:
assert len(response.content) > 1

## Evaluate Customized Model with chat dataset

In [ ]:
# Launch evaluation for customized model
response = client.eval.run_eval(
    benchmark_id=benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": customized_chat_model_dir,
            "sampling_params": {}
        }
    }
)
job_id = response.model_dump()["job_id"]
print(f"Created evaluation job {job_id}")

In [ ]:
print(f"Created evaluation job {job_id}")

In [ ]:
job = wait_eval_job(benchmark_id=benchmark_id, job_id=job_id, polling_interval=5, timeout=600)

In [ ]:
job_results = client.eval.jobs.retrieve(benchmark_id=benchmark_id, job_id=job_id)
print(f"Job results: {json.dumps(job_results.model_dump(), indent=2)}")

In [ ]:
# Extract bleu score and assert it's within range
customized_bleu_score = job_results.scores[benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["bleu"]["scores"]["corpus"]["value"]
print(f"Customized bleu score: {customized_bleu_score}")

assert customized_bleu_score >= 40

In [ ]:
# Extract accuracy and assert it's within range
customized_accuracy_score = job_results.scores[benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["string-check"]["scores"]["string-check"]["value"]
print(f"Customized accuracy: {customized_accuracy_score}")

assert customized_accuracy_score >= 0.47

In [ ]:
# Ensure the customized model evaluation is better than the original model evaluation
print(f"customized_bleu_score - initial_bleu_score: {customized_bleu_score - initial_bleu_score}")
assert (customized_bleu_score - initial_bleu_score) >= 20

print(f"customized_accuracy_score - initial_accuracy_score: {customized_accuracy_score - initial_accuracy_score}")
assert (customized_accuracy_score - initial_accuracy_score) >= 0.2

## Guardrails

We can check messages for safety violations using Guardrails. We'll start by registering a shield for the `llama-3.1-nemoguard-8b-content-safety` model. Ensure the `shield_id` matches the ID of the model we'll use for the safety check.

In [11]:
shield_id = "nvidia/llama-3.1-nemoguard-8b-content-safety"

In [ ]:
client.shields.register(shield_id=shield_id, provider_id="nvidia")

In [ ]:
message = {"role": "user", "content": "You are stupid."}
response = client.safety.run_shield(
    messages=[message],
    shield_id=shield_id,
    params={}
)

print(f"Safety response: {response}")
assert response.violation.user_message == "Sorry I cannot do this."

Guardrails also exposes OpenAI-compatible endpoints you could use to run inference with guardrails.

In [16]:
# Check inference with guardrails
message = {"role": "user", "content": "You are stupid."}
response = requests.post(
    url=f"{NEMO_URL}/v1/guardrail/chat/completions",
    json={
        "model": shield_id,
        "messages": [message],
        "max_tokens": 150
    }
)

assert response.status_code in (200, 201), f"Status Code {response.status_code} Failed to run inference with guardrail {response.text}"

In [ ]:
# Check response contains the predefined message
print(f"Guardrails response: {response.json()['choices'][0]['message']['content']}")
assert response.json()["choices"][0]["message"]["content"] == "I'm sorry, I can't respond to that."

In [ ]:
# Check inference without guardrails
response = client.inference.chat_completion(
    messages=[message],
    model_id=BASE_MODEL,
    sampling_params={
        "max_tokens": 150,
    }
)
assert response.completion_message.content is not None
print(f"Inference response: {response.completion_message.content}")

## Guardrails Evaluation


In [19]:
guardrails_dataset_name = "content-safety-test-data"
guardrails_repo_id = f"{NAMESPACE}/{guardrails_dataset_name}"

In [ ]:
# Create dataset and upload test data
hf_api.create_repo(guardrails_repo_id, repo_type="dataset")
hf_api.upload_folder(
    folder_path="./sample_data/sample_content_safety_test_data",
    path_in_repo="",
    repo_id=guardrails_repo_id,
    repo_type="dataset",
)

In [21]:
guardrails_benchmark_id = "test-guardrails-eval-config"
guardrails_eval_config = {
    "benchmark_id": guardrails_benchmark_id,
    "dataset_id": "",
    "scoring_functions": [],
    "metadata": {
        "type": "custom",
        "params": {"parallelism": 8},
        "tasks": {
            "qa": {
                "type": "completion",
                "params": {
                    "template": {
                        "messages": [
                            {"role": "user", "content": "{{item.prompt}}"},
                        ],
                        "max_tokens": 20,
                        "temperature": 0.7,
                        "top_p": 0.9,
                    },
                },
                "dataset": {"files_url": f"hf://datasets/{guardrails_repo_id}/content_safety_input.jsonl"},
                "metrics": {
                    "bleu": {
                        "type": "bleu",
                        "params": {"references": ["{{item.ideal_response}}"]},
                    },
                },
            }
        }
    }
}

In [ ]:
# Create Evaluation for model, without guardrails. First, register the benchmark.
response = client.benchmarks.register(
    benchmark_id=guardrails_benchmark_id,
    dataset_id=guardrails_repo_id,
    scoring_functions=guardrails_eval_config["scoring_functions"],
    metadata=guardrails_eval_config["metadata"]
)
print(f"Created benchmark {guardrails_benchmark_id}")

In [ ]:
# Run Evaluation for model, without guardrails
response = client.eval.run_eval(
    benchmark_id=guardrails_benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            "model": BASE_MODEL,
            "sampling_params": {}
        }
    }
)
job_id = response.model_dump()["job_id"]
print(f"Created evaluation job {job_id}")

In [ ]:
# Wait for the job to complete
job = wait_eval_job(benchmark_id=guardrails_benchmark_id, job_id=job_id, polling_interval=5, timeout=600)

In [ ]:
print(f"Job {job_id} status: {job.status}")

In [ ]:
job_results = client.eval.jobs.retrieve(benchmark_id=guardrails_benchmark_id, job_id=job_id)
print(f"Job results: {json.dumps(job_results.model_dump(), indent=2)}")

In [ ]:
# Start Evaluation for model, with guardrails
response = requests.post(
    url=f"{NEMO_URL}/v1/evaluation/jobs",
    json={
        "config": guardrails_eval_config["metadata"],
        "target": {
            "type": "model",
            "model": {
                "api_endpoint": {
                    "url": "http://nemo-guardrails:7331/v1/guardrail/completions",
                    "model_id": "meta/llama-3.1-8b-instruct",
                }
            },
        },
    }
)
job_id_with_guardrails = response.json()["id"]
print(f"Created evaluation job with guardrails {job_id_with_guardrails}")

In [ ]:
# Wait for the job to complete
job = wait_eval_job(benchmark_id=guardrails_benchmark_id, job_id=job_id_with_guardrails, polling_interval=5, timeout=600)

In [ ]:
job_results_with_guardrails = client.eval.jobs.retrieve(benchmark_id=guardrails_benchmark_id, job_id=job_id_with_guardrails)
print(f"Job results: {json.dumps(job_results_with_guardrails.model_dump(), indent=2)}")

In [ ]:
bleu_score_no_guardrails = job_results.scores[guardrails_benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["bleu"]["scores"]["corpus"]["value"]
print(f"bleu_score_no_guardrails: {bleu_score_no_guardrails}")

In [ ]:
bleu_score_with_guardrails = job_results_with_guardrails.scores[guardrails_benchmark_id].aggregated_results["tasks"]["qa"]["metrics"]["bleu"]["scores"]["corpus"]["value"]
print(f"bleu_score_with_guardrails: {bleu_score_with_guardrails}")

In [ ]:
# Expect the bleu score to go from 3 to 33
print(f"with_guardrails_bleu_score - no_guardrails_bleu_score: {bleu_score_with_guardrails - bleu_score_no_guardrails}")
assert (bleu_score_with_guardrails - bleu_score_no_guardrails) >= 20

In [ ]:
print("NVIDIA E2E Flow successful.")